![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Legal Data Augmentation with Chunk Mappers

In [0]:
from johnsnowlabs import * 

# About Data Augmentation

__Data Augmentation__ is the process of increase an extracted datapoint with external sources. 

For example, let's suppose I work with a document which mentions the company _Amazon_. We could be talking about stock prices, or some legal litigations, or just a commercial agreement with a provider, among others.

In the document, we can extract `Amazon` using NER as an Organization, but that's all the information available about `Amazon` in that document.

Well, with __Data Augmentation__, we can use external sources, as _SEC Edgar, Crunchbase, Nasdaq_ or even _Wikipedia_, to enrich `Amazon` with much more information, allowing us to take better decisions.

Let's see how to do it.

# Step 1: Name Entity Recognition

Let's suppose we get this news from scrapping the Internet, or from Twitter.

In [0]:
text = "We have entered into a definitive merger agreement with Amazon."

We use NER to extract the companies name, in this case, Amazon.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")
        
sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = legal.NerModel.pretrained("legner_orgs_prods_alias", "en", "legal/models")\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner")
        
ner_converter = nlp.NerConverter()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")

nlpPipeline = nlp.Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][ / ][ — ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
legner_orgs_prods_alias download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

## We use LightPipelines to get the result

In [0]:
lp_ner = nlp.LightPipeline(model)

In [0]:
ner_result = lp_ner.annotate(text)
ner_result

Out[5]: {'document': ['We have entered into a definitive merger agreement with Amazon.'],
 'ner_chunk': ['Amazon'],
 'token': ['We',
 'have',
 'entered',
 'into',
 'a',
 'definitive',
 'merger',
 'agreement',
 'with',
 'Amazon',
 '.'],
 'ner': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O'],
 'embeddings': ['We',
 'have',
 'entered',
 'into',
 'a',
 'definitive',
 'merger',
 'agreement',
 'with',
 'Amazon',
 '.'],
 'sentence': ['We have entered into a definitive merger agreement with Amazon.']}

Alright! Amazon has been detected as an organization. 

Now, let's augment `Amazon` with more information about the company, given that there are no more details in the tweet I can use.

But before __augmenting__, there is a very important step we need to carry out: `Company Name Normalization`

# Step 2: Company Names Normalization

Let's suppose we want to manually get information about Amazon.

Since it's a public US company, we can go to [SEC Edgar's database](https://www.sec.gov/edgar/searchedgar/companysearch) and look for it.

Unfortunately, `Amazon` is not the official name of the company, which means no entry for `Amazon` is available. That's were __Company Names Normalization__ comes in handy.

`Company Name Normalization` is the process of obtaining the name of the company used by data providers, usually the "official" name of the company.

Sometimes, some data providers may have different versions of the name with different punctuation. For example, for Meta:
- Meta Platforms, Inc.
- Meta Platforms Inc.
- Meta Platforms, Inc
- etc

So, it's mandatory we do `Company Normalization` taking into account the database / datasource provider we want to extract data from. The data providers we have are:
- SEC Edgar
- Crunchbase until 2015
- Wikidata (in progress)

Let's normalize `Amazon` to the official name in _SEC Edgar_.

In [0]:
embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")
    
resolver = legal.SentenceEntityResolverModel.pretrained("legel_edgar_company_name", "en", "legal/models")\
      .setInputCols(["text", "sentence_embeddings"]) \
      .setOutputCol("resolution")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = nlp.PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp_res = nlp.LightPipeline(pipelineModel)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
legel_edgar_company_name download started this may take some time.
[ | ][OK!]

In [0]:
ner_result['ner_chunk']

Out[7]: ['Amazon']

In [0]:
el_res = lp_res.annotate(ner_result['ner_chunk'])
el_res

Out[8]: [{'document': ['Amazon'],
 'sentence_embeddings': ['Amazon'],
 'resolution': ['AMAZON COM INC']}]

Here is our normalized name for Amazon: `AMAZON COM INC`.

Now, let's see which information is available in Edgar database for `AMAZON COM INC`

# Steps 1 and 2 in the same pipeline

In [0]:
documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")
        
sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = legal.NerModel.pretrained("legner_orgs_prods_alias", "en", "legal/models")\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner")
        
ner_converter = nlp.NerConverter()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")

chunk2doc = nlp.Chunk2Doc()\
        .setInputCols("ner_chunk")\
        .setOutputCol("ner_chunk_doc")

sentence_embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("ner_chunk_doc") \
      .setOutputCol("sentence_embeddings")
    
resolver = legal.SentenceEntityResolverModel.pretrained("legel_edgar_company_name", "en", "legal/models")\
      .setInputCols(["text", "sentence_embeddings"]) \
      .setOutputCol("resolution")\
      .setDistanceFunction("EUCLIDEAN")

nlpPipeline = nlp.Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
        chunk2doc,
        sentence_embeddings,
        resolver
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
legner_orgs_prods_alias download started this may take some time.
[ | ][OK!]
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
legel_edgar_company_name download started this may take some time.
[ | ][OK!]

In [0]:
lp_model = nlp.LightPipeline(model)

In [0]:
el_res = lp_model.annotate(text)
el_res

Out[11]: {'document': ['We have entered into a definitive merger agreement with Amazon.'],
 'ner_chunk': ['Amazon'],
 'sentence_embeddings': ['Amazon'],
 'resolution': ['AMAZON COM INC'],
 'token': ['We',
 'have',
 'entered',
 'into',
 'a',
 'definitive',
 'merger',
 'agreement',
 'with',
 'Amazon',
 '.'],
 'ner': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O'],
 'embeddings': ['We',
 'have',
 'entered',
 'into',
 'a',
 'definitive',
 'merger',
 'agreement',
 'with',
 'Amazon',
 '.'],
 'ner_chunk_doc': ['Amazon'],
 'sentence': ['We have entered into a definitive merger agreement with Amazon.']}

# Step 3: Data Augmentation with Chunk Mappers

The component which carries out __Data Augmentation__ is called `ChunkMapper`.

It's name comes from the way it works: it uses a _Ner Chunk_ to map it to an external data source.

As a result, you will get a JSON with a dictionary of additional fields and their values. 

Let's take a look at how it works.

In [0]:
chunkAssembler = nlp.Doc2Chunk() \
    .setInputCols("document") \
    .setOutputCol("chunk") \
    .setIsArray(False)

CM =legal.ChunkMapperModel().pretrained("legmapper_edgar_companyname", "en", "legal/models")\
      .setInputCols(["chunk"])\
      .setOutputCol("mappings")

cm_pipeline = nlp.Pipeline(stages=[documentAssembler, chunkAssembler, CM])
fit_cm_pipeline = cm_pipeline.fit(empty_data)

legmapper_edgar_companyname download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
# LightPipelines don't support Doc2Chunk, so we will use here usual transform

df = spark.createDataFrame([el_res['resolution']]).toDF("text")
df.show()

+--------------+
 text|
+--------------+
AMAZON COM INC|
+--------------+

In [0]:
res = fit_cm_pipeline.transform(df)
res.show()

+--------------+--------------------+--------------------+--------------------+
 text| document| chunk| mappings|
+--------------+--------------------+--------------------+--------------------+
AMAZON COM INC|[{document, 0, 13...|[{chunk, 0, 13, A...|[{labeled_depende...|
+--------------+--------------------+--------------------+--------------------+

In [0]:
r = res.collect()
r

Out[15]: [Row(text='AMAZON COM INC', document=[Row(annotatorType='document', begin=0, end=13, result='AMAZON COM INC', metadata={'sentence': '0'}, embeddings=[])], chunk=[Row(annotatorType='chunk', begin=0, end=13, result='AMAZON COM INC', metadata={'sentence': '0', 'chunk': '0'}, embeddings=[])], mappings=[Row(annotatorType='labeled_dependency', begin=0, end=13, result='AMAZON COM INC', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'name', 'all_relations': ''}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'sic', 'all_relations': '[5961'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='5961', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'sic_code', 'all_relations': '0'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='911646860', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'irs_number', 'all_relations': '0:::261631624'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='1231', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'fiscal_year_end', 'all_relations': '0'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='WA', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'state_location', 'all_relations': ''}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='DE', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'state_incorporation', 'all_relations': ''}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='410 TERRY AVENUE NORTH', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'business_street', 'all_relations': '1200 12TH AVENUE S SUITE 1200'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='SEATTLE', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'business_city', 'all_relations': ''}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='WA', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'business_state', 'all_relations': ''}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='98109', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'business_zip', 'all_relations': '98144'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='2062661000', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'business_phone', 'all_relations': ''}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='ABX Holdings, Inc.', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'former_name', 'all_relations': 'ABX AIR INC'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='20080102', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'former_name_date', 'all_relations': '19950728'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=13, result='2017-02-10', metadata={'sentence': '0', 'chunk': '0', 'entity': 'AMAZON COM INC', 'relation': 'date', 'all_relations': '2016-01-29:::2016-02-10:::2016-09-08:::2016-10-28:::2019-02-01:::2019-10-25:::2018-02-02:::2018-07-27:::2015-01-30:::2015-07-24:::2015-10-23:::2015-12-04:::2014-01-31:::2014-01-30:::2014-02-11:::2014-02-18:::2014-02-19:::2014-02-13:::2014-02-20:::2014-03-06:::2014-04-09:::2014-04-04:::2013-01-30:::2012-02-01:::2011-01-28:::2011-07-27:::2011-10-26:::2022-02-04:::2022-07-29:::2021-02-03:::2021-04-08:::2020-01-31:::2020-05-01:::2020-10-30:::2

In [0]:
json_dict = dict()
for n in r[0]['mappings']:
    json_dict[n.metadata['relation']] = str(n.result)

In [0]:
import json
print(json.dumps(json_dict, indent=4, sort_keys=True))

{
 "business_city": "SEATTLE",
 "business_phone": "2062661000",
 "business_state": "WA",
 "business_street": "410 TERRY AVENUE NORTH",
 "business_zip": "98109",
 "company_id": "1018724",
 "date": "2017-02-10",
 "fiscal_year_end": "1231",
 "former_name": "ABX Holdings, Inc.",
 "former_name_date": "20080102",
 "irs_number": "911646860",
 "name": "AMAZON COM INC",
 "sic": "RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]",
 "sic_code": "5961",
 "state_incorporation": "DE",
 "state_location": "WA"
}